# IBM Applied Data Science Capstone Course by Coursera
Week 5 Final Report
Opening a New Shopping Mall in Kuala Lumpur, Malaysia

Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighborhoods
Select the best cluster to open a new shopping mall

1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
! pip install geocoder
import geocoder # to get coordinates
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
kaulalumpur_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kaulalumpur_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [7]:
# print the number of rows of the dataframe
kaulalumpur_df.shape

(71, 1)

3. Get the geographical coordinates

In [8]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in kaulalumpur_df["Neighborhood"].tolist() ]
coordinates

[[3.0576900000000364, 101.74388000000005],
 [3.148494115588384, 101.69672876508707],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1511418740671178, 101.69937468268016],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.136442497863743, 101.69029641203808],
 [3.1419057131529953, 101.67967833034047],
 [3.1566851613075664, 101.6

In [10]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coordinates= pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [11]:
# merge the coordinates into the original dataframe
kaulalumpur_df['Latitude'] = df_coordinates['Latitude']
kaulalumpur_df['Longitude'] = df_coordinates['Longitude']

In [12]:
# check the neighborhoods and the coordinates
print(kaulalumpur_df.shape)
kaulalumpur_df.head()

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [13]:
# save the DataFrame as CSV file
kaulalumpur_df.to_csv("kaulalumpur_df.csv", index=False)

## 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [14]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysia 3.1516964, 101.6942371.


In [15]:
# create map of Kaula Lumpur using latitude and longitude values
map_kaulalumpur = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kaulalumpur_df['Latitude'], kaulalumpur_df['Longitude'], kaulalumpur_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kaulalumpur)  
    
map_kaulalumpur

In [16]:
# save the map as HTML file
map_kaulalumpur.save('map_kaulalumpur.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [17]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'O2OS3TBT0KFW2P3P0RTETDSDJBL4UGOE5VAHV5YI2A5RKZGE' # your Foursquare ID
CLIENT_SECRET = 'HFN05PEOBSVV1ZN0NQJKPNDAV1CSRZROFDCJ5X2ULUOTNHSF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O2OS3TBT0KFW2P3P0RTETDSDJBL4UGOE5VAHV5YI2A5RKZGE
CLIENT_SECRET:HFN05PEOBSVV1ZN0NQJKPNDAV1CSRZROFDCJ5X2ULUOTNHSF


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
kaulalumpur_venues = getNearbyVenues(names=kaulalumpur_df['Neighborhood'],
                                   latitudes=kaulalumpur_df['Latitude'],
                                   longitudes=kaulalumpur_df['Longitude'])

Alam Damai
Ampang, Kuala Lumpur
Bandar Menjalara
Bandar Sri Permaisuri
Bandar Tasik Selatan
Bandar Tun Razak
Bangsar
Bangsar Park
Bangsar South
Batu 11 Cheras
Batu, Kuala Lumpur
Brickfields
Bukit Bintang
Bukit Jalil
Bukit Kiara
Bukit Nanas
Bukit Petaling
Bukit Tunku
Cheras, Kuala Lumpur
Chow Kit
Damansara Heights
Damansara Town Centre
Damansara, Kuala Lumpur
Dang Wangi
Desa Petaling
Federal Hill, Kuala Lumpur
Happy Garden
Jalan Cochrane, Kuala Lumpur
Jalan Duta
Jinjang
Kampung Baru, Kuala Lumpur
Kampung Datuk Keramat
Kampung Padang Balang
Kepong
Kepong Baru
KL Eco City
Kuchai Lama
Lembah Pantai
Maluri
Medan Tuanku
Miharja
Mont Kiara
Pantai Dalam
Pudu, Kuala Lumpur
Putrajaya
Salak South
Segambut
Semarak
Sentul, Kuala Lumpur
Setapak
Setiawangsa
Shamelin
Sri Hartamas
Sri Petaling
Sungai Besi
Taman Bukit Maluri
Taman Connaught
Taman Desa
Taman Ibukota
Taman Len Seng
Taman Melati
Taman Midah
Taman OUG
Taman P. Ramlee
Taman Sri Sinar
Taman Taynton View
Taman Tun Dr Ismail
Taman U-Thant
Taman

In [20]:
kaulalumpur_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
1,Alam Damai,3.05769,101.74388,閒茶素食店 Leisure Tea Vegetarian,3.057673,101.747258,Vegetarian / Vegan Restaurant
2,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
3,Alam Damai,3.05769,101.74388,Roti Canai Perindu,3.055071,101.740574,Breakfast Spot
4,Alam Damai,3.05769,101.74388,Heng Kee Tim Sum 興記點心,3.057513,101.747191,Dim Sum Restaurant


Let's check how many venues were returned for each neighorhood

In [21]:
kaulalumpur_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alam Damai,19,19,19,19,19,19
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,51,51,51,51,51,51
Bandar Sri Permaisuri,40,40,40,40,40,40
Bandar Tasik Selatan,35,35,35,35,35,35
Bandar Tun Razak,13,13,13,13,13,13
Bangsar,28,28,28,28,28,28
Bangsar Park,28,28,28,28,28,28
Bangsar South,69,69,69,69,69,69


Let's find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(kaulalumpur_venues['Venue Category'].unique())))

There are 281 uniques categories.


In [23]:
# print out the list of categories
kaulalumpur_venues['Venue Category'].unique()[:50]

array(['Noodle House', 'Vegetarian / Vegan Restaurant',
       'Chinese Restaurant', 'Breakfast Spot', 'Dim Sum Restaurant',
       'Dessert Shop', 'Farmers Market', 'Convenience Store',
       'Japanese Restaurant', 'Hakka Restaurant', 'Pharmacy',
       'Fast Food Restaurant', 'Food Court', 'Asian Restaurant',
       'Food Truck', 'Padangnese Restaurant', 'Hostel',
       'South Indian Restaurant', 'Exhibit', 'Chettinad Restaurant',
       'Indian Restaurant', 'Monument / Landmark', 'Park', 'Hotel',
       'Athletics & Sports', 'Restaurant', 'Gym', 'Café', 'Coffee Shop',
       'Juice Bar', 'Theater', 'Bakery', 'Bridal Shop', 'Flower Shop',
       'Art Gallery', 'Food Stand', 'Wine Bar', 'Flea Market', 'Bar',
       'Bed & Breakfast', 'Thai Restaurant', 'Filipino Restaurant',
       'Cupcake Shop', 'Malay Restaurant', 'Other Nightlife',
       'Music Venue', 'Sandwich Place', 'Bubble Tea Shop',
       'Clothing Store', 'Deli / Bodega'], dtype=object)

In [24]:
# check if the results contain "Restaurant"
"Neighborhood" in kaulalumpur_venues['Venue Category'].unique()

True

6. Analyze Each Neighborhood

In [25]:
# one hot encoding
kaulalumpur_onehot = pd.get_dummies(kaulalumpur_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kaulalumpur_onehot['Neighborhoods'] = kaulalumpur_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kaulalumpur_onehot.columns[-1]] + list(kaulalumpur_onehot.columns[:-1])
kaulalumpur_onehot = kaulalumpur_onehot[fixed_columns]

print(kaulalumpur_onehot.shape)
kaulalumpur_onehot.head()

(2669, 282)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Food Court,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Belgian Restaurant,Betting Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyb

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
kaulalumpur_grouped = kaulalumpur_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kaulalumpur_grouped.shape)
kaulalumpur_grouped

(71, 282)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Food Court,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Belgian Restaurant,Betting Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyb

In [27]:
len(kaulalumpur_grouped[kaulalumpur_grouped["Asian Restaurant"] > 0])

50

Create a new DataFrame for Shopping Mall data only

In [28]:
kaulalumpur_ar = kaulalumpur_grouped[["Neighborhoods","Asian Restaurant"]]
kaulalumpur_ar.head()

,Neighborhoods,Asian Restaurant
0,Alam Damai,0.052632
1,"Ampang, Kuala Lumpur",0.050000
2,Bandar Menjalara,0.058824
3,Bandar Sri Permaisuri,0.000000
4,Bandar Tasik Selatan,0.171429


## 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [29]:
# set number of clusters
kclusters = 3

kaulalumpur_clustering =kaulalumpur_ar.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kaulalumpur_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [30]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kaulalumpur_merged = kaulalumpur_ar.copy()

# add clustering labels
kaulalumpur_merged["Cluster Labels"] = kmeans.labels_

In [31]:
kaulalumpur_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kaulalumpur_merged.head()

,Neighborhood,Asian Restaurant,Cluster Labels
0,Alam Damai,0.052632,2
1,"Ampang, Kuala Lumpur",0.050000,2
2,Bandar Menjalara,0.058824,2
3,Bandar Sri Permaisuri,0.000000,1
4,Bandar Tasik Selatan,0.171429,0


In [32]:
# merge Kaulalumpur_grouped with Kaulalumpur_data to add latitude/longitude for each neighborhood
kaulalumpur_merged = kaulalumpur_merged.join(kaulalumpur_df.set_index("Neighborhood"), on="Neighborhood")

print(kaulalumpur_merged.shape)
kaulalumpur_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.052632,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.050000,2,3.148494,101.696729
2,Bandar Menjalara,0.058824,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.000000,1,3.103910,101.712260
4,Bandar Tasik Selatan,0.171429,0,3.072750,101.714610


In [33]:
# sort the results by Cluster Labels
print(kaulalumpur_merged.shape)
kaulalumpur_merged.sort_values(["Cluster Labels"], inplace=True)
kaulalumpur_merged

(71, 5)


,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
35,Kepong Baru,0.121212,0,3.207768,101.645167
29,Jinjang,0.176471,0,3.209500,101.658740
65,Taman Taynton View,0.172414,0,3.087070,101.736810
4,Bandar Tasik Selatan,0.171429,0,3.072750,101.714610
63,Taman P. Ramlee,0.222222,0,3.193940,101.705730
62,Taman OUG,0.178571,0,3.210050,101.634508
47,Semarak,0.125000,0,3.179916,101.721437
30,KL Eco City,0.010000,1,3.117130,101.673840
32,Kampung Datuk Keramat,0.021739,1,3.166400,101.730460
42,Pantai Dalam,0.000000,1,3.094760,101.667470


Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kaulalumpur_merged['Latitude'], kaulalumpur_merged['Longitude'], kaulalumpur_merged['Neighborhood'], kaulalumpur_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

8. Examine Clusters
Cluster 0

In [36]:
kaulalumpur_merged.loc[kaulalumpur_merged['Cluster Labels'] == 0]

,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
35,Kepong Baru,0.121212,0,3.207768,101.645167
29,Jinjang,0.176471,0,3.209500,101.658740
65,Taman Taynton View,0.172414,0,3.087070,101.736810
4,Bandar Tasik Selatan,0.171429,0,3.072750,101.714610
63,Taman P. Ramlee,0.222222,0,3.193940,101.705730
62,Taman OUG,0.178571,0,3.210050,101.634508
47,Semarak,0.125000,0,3.179916,101.721437


Cluster 1

In [37]:
kaulalumpur_merged.loc[kaulalumpur_merged['Cluster Labels'] == 1]

,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
30,KL Eco City,0.010000,1,3.117130,101.673840
32,Kampung Datuk Keramat,0.021739,1,3.166400,101.730460
42,Pantai Dalam,0.000000,1,3.094760,101.667470
43,"Pudu, Kuala Lumpur",0.022727,1,3.133540,101.713070
45,Salak South,0.000000,1,3.081020,101.697240
46,Segambut,0.000000,1,3.186390,101.668100
55,Taman Bukit Maluri,0.000000,1,3.200660,101.633370
52,Sri Hartamas,0.030000,1,3.162200,101.650360
28,Jalan Duta,0.000000,1,3.180025,101.677834
57,Taman Desa,0.000000,1,3.102970,101.684710


Cluster 2

In [38]:
kaulalumpur_merged.loc[kaulalumpur_merged['Cluster Labels'] == 2]

,Neighborhood,Asian Restaurant,Cluster Labels,Latitude,Longitude
38,Maluri,0.051282,2,3.147890,101.694050
68,Taman Wahyu,0.058824,2,3.222400,101.671730
1,"Ampang, Kuala Lumpur",0.050000,2,3.148494,101.696729
2,Bandar Menjalara,0.058824,2,3.190350,101.625450
26,Happy Garden,0.040000,2,3.201630,101.721070
27,"Jalan Cochrane, Kuala Lumpur",0.058824,2,3.132903,101.724679
60,Taman Melati,0.050000,2,3.223570,101.723990
31,"Kampung Baru, Kuala Lumpur",0.081081,2,3.165460,101.710280
58,Taman Ibukota,0.045455,2,3.212160,101.715400
10,"Batu, Kuala Lumpur",0.051282,2,3.147890,101.694050


### Observations:
Most of the Asian restaurants are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has very low number of restaurants in the neighborhoods. This represents a great opportunity and high potential areas to open new restaurants as there is very little to no competition from existing restaurants. Meanwhile, restaurants in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of restaurants. From another perspective, this also shows that the oversupply of restaurants mostly happened in the central area of the city, with the suburb area still have very few restaurants. Therefore, this project recommends property developers to capitalize on these findings to open new restaurants in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new restaurants in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of restaurants and suffering from intense competition.